In [1]:
from stage1 import *

In [2]:
nframes,natoms,nmols=(0,0,0)

In [3]:
from stage2 import * 

In [4]:
stage1

VGAE(
  (encoder): VariationalGCNEncoder(
    (conv1): GATConv(5, 15, heads=3)
    (head_transform1): Linear(45, 15, bias=True)
    (bn1): BatchNorm(15)
    (conv2): GCNConv(15, 9)
    (bn2): BatchNorm(9)
    (linear1): Linear(9, 100, bias=True)
    (linear2): Linear(100, 4, bias=True)
    (transform): Linear(16, 3, bias=True)
    (mu): Linear(3, 3, bias=True)
    (logstd): Linear(3, 3, bias=True)
  )
  (decoder): VariationalGCNDecoder(
    (inv_transform): Linear(3, 4, bias=True)
    (conv1): GCNConv(1, 9)
    (bn1): BatchNorm(9)
    (conv2): GCNConv(9, 3)
    (bn2): BatchNorm(3)
    (conv3): GCNConv(3, 3)
    (linear1): Linear(3, 512, bias=True)
    (linear2): Linear(512, 128, bias=True)
    (linear3): Linear(128, 5, bias=True)
  )
)

In [5]:
testframe=md.load_xtc("../../xtc_files/50000eql2.xtc",top="../../xtc_files/conf.gro",frame=0)


In [6]:
cf=condenseframe(testframe)

In [7]:
meltframes=md.load_xtc("../../xtc_files/50000eql2.xtc",top="../../xtc_files/conf.gro")
meltframes.xyz=meltframes.xyz*10
global nframes,natoms,nmols 
nframes = meltframes.xyz.shape[0]
natoms = meltframes.topology.residue(0).n_atoms
nmols = meltframes.topology.n_residues
print (nframes,natoms,nmols)
meltframes=meltframes[::1000]

50001 4 592


In [8]:
cf

array([[ 7.84576243e+02, -9.36140506e+01, -1.56635818e+02,
         2.28225017e+00,  9.52250063e-01,  1.95350003e+00],
       [ 7.79803099e+02, -9.30955292e+01, -1.55628271e+02,
         1.36549997e+00,  1.45650005e+00,  2.59900022e+00],
       [ 8.75985383e+02, -1.05724399e+02, -1.74737958e+02,
         8.33750010e-01,  1.77175009e+00,  7.37500072e-01],
       ...,
       [ 7.41200941e+02, -8.83453476e+01, -1.47991849e+02,
         2.56474996e+00,  1.57500005e+00,  2.47574997e+00],
       [ 7.57661722e+02, -9.03470056e+01, -1.51268077e+02,
         1.61825013e+00,  1.92200017e+00,  2.49900007e+00],
       [ 8.03441321e+02, -9.64539095e+01, -1.60332419e+02,
         7.91500092e-01,  2.24175000e+00,  2.16525006e+00]])

In [9]:
melt_condensed=condenseAllFrames(meltframes)
graphs_melt_cpu=get_graphs(melt_condensed,"melt")

100%|██████████| 51/51 [00:01<00:00, 27.83it/s]


In [10]:
torch.save(graphs_melt_cpu,f'./graphs/melt_neigh{n_neigh}.pt')

In [11]:
g = get_graph(cf,1,n_neigh,"cry")
# print(g.x)

In [12]:
graphs_cpu=graphs_melt_cpu
np.random.shuffle(graphs_cpu)


In [22]:
import os
in_channels = graphs_cpu[0].num_features
out_channels = 5
lr = 1e-4
n_epochs = 400
batch_size=64
test_train_split = 0.8
model_name = "Inter.pt"
model_loaded = False
force_train = True
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

if os.path.exists("./"+model_name) and not force_train:
    model = torch.load("./"+model_name)
    model_loaded = True
else:
    model = VGAE_S2(VariationalGCNEncoder_S2(in_channels, out_channels,batch_size,n_neigh+1),
                    VariationalGCNDecoder_S2(out_channels, in_channels,batch_size,n_neigh+1))

model = model.to(device)
model = model.double()
# optimizer = torch.optim.SGD(model.parameters(), lr=0.1, momentum=0.9)

optimizer = torch.optim.Adam(model.parameters(), lr=lr)
scheduler = lr_scheduler.StepLR(optimizer, step_size=300, gamma=0.5)


In [23]:
graphs_device = []
for graph in tqdm(graphs_cpu):
    graphs_device.append(graph.to(device))

100%|██████████| 30192/30192 [00:00<00:00, 57752.63it/s]


In [24]:
graphs = graphs_device
total_graphs = len(graphs)


In [25]:
print("Using:",device )
print("total_graphs:", total_graphs)
print("batch Size:" , batch_size)
print("number of batches:",total_graphs//batch_size)
print("Neighbours:",n_neigh)
print("Model\n",model)

Using: cuda
total_graphs: 30192
batch Size: 64
number of batches: 471
Neighbours: 15
Model
 VGAE_S2(
  (encoder): VariationalGCNEncoder_S2(
    (conv1): GATConv(6, 20, heads=6)
    (head_transform1): Linear(120, 20, bias=True)
    (bn1): BatchNorm(20)
    (conv2): GATConv(20, 10, heads=6)
    (head_transform2): Linear(60, 10, bias=True)
    (bn2): BatchNorm(10)
    (linear1): Linear(10, 100, bias=True)
    (linear2): Linear(100, 4, bias=True)
    (transform): Linear(64, 5, bias=True)
    (mu): Linear(5, 5, bias=True)
    (logstd): Linear(5, 5, bias=True)
  )
  (decoder): VariationalGCNDecoder_S2(
    (inv_transform): Linear(5, 80, bias=True)
    (conv1): GCNConv(5, 3)
    (conv2): GCNConv(3, 10)
    (conv3): GCNConv(10, 30)
    (bn): BatchNorm(30)
    (linear1): Linear(30, 128, bias=True)
    (linear2): Linear(128, 64, bias=True)
    (linear3): Linear(64, 6, bias=True)
  )
)


In [ ]:
split = int(test_train_split * total_graphs)

train_loader = DataLoader(graphs[:split], batch_size=batch_size, shuffle=True)
test_loader = DataLoader(graphs[split:], batch_size=batch_size, shuffle=True)

In [26]:
cross_entropy_loss = torch.nn.CrossEntropyLoss()
MSE_loss = torch.nn.MSELoss()

def train():
    model.train()
    model.zero_grad()
    all_loss_mse = 0
    for data in train_loader:
        optimizer.zero_grad()
        
        z,encoded_edge_index = model.encode(data.x, data.edge_index)
        decoded_x, decoded_edge_index = model.decode(z, encoded_edge_index)

        
        loss = MSE_loss(decoded_x, data.x)


        loss.backward()
        all_loss_mse += float(loss)
        optimizer.step()


    return all_loss_mse/len(train_loader)

In [27]:
def test():
    model.eval()
    model.zero_grad()
    all_loss_mse = 0
    for data in test_loader:
        optimizer.zero_grad()
        z,encoded_edge_index = model.encode(data.x, data.edge_index)
        decoded_x, decoded_edge_index = model.decode(z, encoded_edge_index)

        loss = MSE_loss(decoded_x, data.x)

        all_loss_mse += float(loss)


    return all_loss_mse/len(test_loader)

In [29]:
train_losses = []
test_losses = []

if model_loaded:
    print("Pretrained Model Loaded, no training required")
else:
    for epoch in range(1, n_epochs + 1):

        train_loss_mse = train()
        test_loss_mse = test()

        print(f"Epoch: {epoch:03d}")
        print(f'\tTrain:\tMSE Loss: {train_loss_mse:.4f}')
        print(f'\tTest: \tMSE Loss: {test_loss_mse:.4f}')

        train_losses.append([train_loss_mse]);
        test_losses.append([test_loss_mse]);

        scheduler.step()
    torch.save(model,"./"+model_name)

KeyboardInterrupt: 

In [ ]:
if not model_loaded:
    plt.figure()
    plt.plot(test_losses, label='Total Test Loss')
    plt.plot(train_losses, label='Total Train Loss')
    plt.legend()
    plt.title('Total Loss')
    plt.show()

In [ ]:
test_loader_melt = DataLoader(graphs_melt_cpu[::100], batch_size=1)
pred_melt=[]
for data in tqdm(test_loader_melt):
    model.eval()
    data = data.to(device)
    pred = model(data.x.float(),data.edge_index)
    pred_melt.append(pred.cpu().detach().numpy())
pred_melt = np.array(pred_melt)


In [ ]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

y_test = test_loader_melt
cm = confusion_matrix(y_test, predictions)
ConfusionMatrixDisplay(cm).plot()


In [ ]:
# %matplotlib widget

fig = plt.figure(figsize=(15,4))
data_melt = pred_melt
data_cry = pred_cry

plt.scatter(data_melt[:,0],data_melt[:,1],c="r",label="melt")
plt.scatter(data_cry[:,0],data_cry[:,1],c="b",label="crystal")
plt.legend()
plt.show()